In [1]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [2]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Dot Product

In [3]:
last_skywalker = np.array([0.98, 0.9, -0.9])

In [4]:
user1 = np.array([0.9, 0.8, -0.6])

In [5]:
(user1*last_skywalker).sum()

2.1420000000000003

In [6]:
casablanca = np.array([-0.99, -0.3, 0.8])

In [7]:
(user1*casablanca).sum()

-1.611

# Creating the DataLoaders

In [8]:
movies = pd.read_csv(path/'u.item', delimiter="|", encoding='latin-1',
                    usecols=(0,1), names=('movie','title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [9]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [10]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,286,Fantasia (1940),3
1,318,D3: The Mighty Ducks (1996),4
2,393,Beverly Hills Ninja (1997),3
3,332,Die Hard 2 (1990),5
4,180,Philadelphia (1993),4
5,378,Menace II Society (1993),4
6,537,Hard Rain (1998),1
7,94,Dead Man Walking (1995),5
8,137,Evita (1996),3
9,682,Hoop Dreams (1994),4


In [11]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
# latent factors
n_factors = 5
n_users, n_movies

(944, 1665)

In [12]:
user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [13]:
one_hot_2 = one_hot(2, n_users).float()
#one_hot_2 @ user_factors
user_factors.t()@one_hot_2

tensor([ 0.2116,  1.4282,  1.2174,  0.9891, -0.2229])

In [14]:
user_factors.t()

tensor([[-1.3607,  1.8799,  0.2116,  ...,  0.3090,  0.4712,  1.3124],
        [ 1.6060, -0.2009,  1.4282,  ..., -0.5632,  1.4788,  0.5509],
        [ 0.5357,  1.1350,  1.2174,  ..., -0.4852,  0.0866,  0.2778],
        [ 0.4846,  1.3070,  0.9891,  ...,  0.3884,  0.2728, -0.9251],
        [ 1.9805, -2.4303, -0.2229,  ..., -0.5519, -2.9007, -0.7269]])

In [15]:
user_factors.shape

torch.Size([944, 5])

In [16]:
one_hot_2[0:9]

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [17]:
class Example:
    def __init__(self, a): self.a = a
    def say(self, x): return f'Hello {self.a}, {x}.'

In [18]:
ex = Example('Sylvain')
ex.say('nice to meet you')

'Hello Sylvain, nice to meet you.'

In [19]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return (users * movies).sum(dim=1)

In [20]:
x, y = dls.one_batch()
x.shape

torch.Size([64, 2])

In [21]:
y.shape

torch.Size([64, 1])

In [22]:
# em_users = Embedding(n_users, n_factors)
# em_users

In [23]:
# em_movies = Embedding(n_movies, n_factors)
# em_movies

In [24]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [25]:
# learn.fit_one_cycle(5, 5e-3)

In [26]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.y_range = y_range
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return sigmoid_range((users * movies).sum(dim=1), *self.y_range)

In [27]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.986631,1.001518,00:05
1,0.865788,0.904361,00:05
2,0.689167,0.858004,00:05
3,0.481246,0.861733,00:05
4,0.359776,0.865496,00:05


In [52]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)
        self.y_range = y_range
    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)

In [29]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.928735,0.930388,00:05
1,0.851410,0.854446,00:05
2,0.582623,0.858356,00:05
3,0.397896,0.878850,00:05
4,0.295188,0.885508,00:05


# Testing SUM DIM

In [30]:
mat = torch.arange(9).view(3, -1)
mat

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [31]:
torch.sum(mat, dim=0)

tensor([ 9, 12, 15])

In [32]:
torch.sum(mat, dim=1)

tensor([ 3, 12, 21])

In [33]:
rank_3_tensor = torch.tensor([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])


In [34]:
rank_3_tensor.sum(dim=0)

tensor([[30, 33, 36, 39, 42],
        [45, 48, 51, 54, 57]])

In [35]:
rank_3_tensor.sum(dim=1)

tensor([[ 5,  7,  9, 11, 13],
        [25, 27, 29, 31, 33],
        [45, 47, 49, 51, 53]])

In [36]:
rank_3_tensor.sum(dim=2)

tensor([[ 10,  35],
        [ 60,  85],
        [110, 135]])

In [37]:
rank_3_tensor.sum(dim=-1)

tensor([[ 10,  35],
        [ 60,  85],
        [110, 135]])

In [38]:
rank_3_tensor.sum(dim=-2)

tensor([[ 5,  7,  9, 11, 13],
        [25, 27, 29, 31, 33],
        [45, 47, 49, 51, 53]])

In [39]:
rank_3_tensor.sum(dim=-3)

tensor([[30, 33, 36, 39, 42],
        [45, 48, 51, 54, 57]])

# Crosstab with pandas

In [40]:
# pd.crosstab(ratings['user'], ratings['movie'], ratings['rating'], aggfunc='mean')

In [41]:
n_users = 10
n_movies = 20

# list the ids of the most frequent users (those who rated the most movies)
most_frequent_users = list(ratings.user.value_counts().index[:n_users])
# grab all the ratings made by these most frequent users
denser_users = ratings[ratings.user.isin(most_frequent_users)]

# list the ids of the most frequent movies within this group of users
dense_users_most_rated = list(ratings.movie.value_counts().index[:n_movies])
# grab all the most frequent movies rated by the most frequent users
denser_movies = ratings[ratings.movie.isin(dense_users_most_rated)]

# plot the crosstab
pd.crosstab(denser_users.user, denser_movies.movie, values=ratings.rating, aggfunc='mean').fillna('-')


movie,1,7,50,56,98,100,117,121,127,172,174,181,222,237,258,286,288,294,300,313
user,,,,,,,,,,,,,,,,,,,,
13,3,2,5.0,5.0,4,5,3,5,5,5.0,4,5.0,3,5,4,3,1.0,2,1,4
234,3,2,4.0,3.0,4,4,2,-,4,3.0,3,3.0,3,3,2,3,3.0,3,3,4
276,5,5,5.0,5.0,5,5,4,4,5,5.0,5,5.0,4,5,5,-,4.0,4,4,5
303,5,4,5.0,5.0,5,5,3,3,5,5.0,5,5.0,3,5,4,5,4.0,4,1,-
393,3,4,5.0,2.0,-,1,4,4,-,5.0,-,4.0,4,4,4,-,3.0,4,-,4
405,-,-,5.0,4.0,4,-,-,-,5,5.0,5,5.0,-,-,-,-,5.0,-,-,4
416,5,4,5.0,5.0,5,5,5,5,5,5.0,5,5.0,-,3,5,5,5.0,4,4,5
450,4,4,5.0,4.0,4,4,4,3,5,4.0,5,4.0,3,5,4,4,3.0,4,4,5
537,2,4,4.0,5.0,3,4,2,1,5,3.0,3,2.0,2,3,4,3,2.0,1,1,4


# From QL6

In [42]:
a = [1, 2, 3, 4]
b = [5, 6, 7, 8]
sum(i[0]*i[1] for i in zip(a,b))

70

In [43]:
1*5+2*6+3*7+4*8

70

# Weight Decay

In [44]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.411866,0.889950,00:05
1,0.558969,0.876326,00:05
2,0.481189,0.862429,00:05
3,0.351504,0.860895,00:05
4,0.260382,0.863171,00:05


# Using fastai.collab

In [58]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))

In [59]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.947309,0.938127,00:06
1,0.843582,0.859545,00:06
2,0.733909,0.827908,00:06
3,0.590980,0.808683,00:05
4,0.490034,0.809969,00:05


In [60]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

In [61]:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Titanic (1997)',
 'Shawshank Redemption, The (1994)',
 "Schindler's List (1993)",
 'As Good As It Gets (1997)',
 'Good Will Hunting (1997)']

In [63]:
movie_factors = learn.model.i_weight.weight
idx = dls.classes['title'].o2i['Silence of the Lambs, The (1991)']
distances = nn.CosineSimilarity(dim=1)(movie_factors, movie_factors[idx][None])
idx = distances.argsort(descending=True)[1]
dls.classes['title'][idx]

"One Flew Over the Cuckoo's Nest (1975)"

In [65]:
embs = get_emb_sz(dls)
embs

[(944, 74), (1665, 102)]

In [66]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
        nn.Linear(user_sz[1]+item_sz[1], n_act),
        nn.ReLU(),
        nn.Linear(n_act, 1))
        self.y_range = y_range
    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

In [67]:
model = CollabNN(*embs)

In [68]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.948795,0.945481,00:08
1,0.882849,0.899279,00:06
2,0.851647,0.889094,00:06
3,0.827965,0.867500,00:06
4,0.788890,0.868754,00:06


In [69]:
learn = collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=[100,50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.981210,0.990804,00:09
1,0.900186,0.931429,00:07
2,0.844837,0.890627,00:07
3,0.832257,0.861363,00:07
4,0.789651,0.860592,00:07
